# San Francisco Businesses: Boom or Bust?

By Hannah Bassett and Hannah Freitag 

### DATA ANALYSIS

### The Project

The purpose of this project is to identify changes that occurred among San Francisco’s businesses during the COVID-19 pandemic. Our analysis of DataSF’s [Registered Business Locations]("https://data.sfgov.org/Economy-and-Community/Registered-Business-Locations-San-Francisco/g8m3-pdis) dataset found that overall the number of business openings have decreased over the past five years–a trend that is reflected among restaurant openings, as well. The Financial District/South Beach experienced the most business openings, but the annual number of restaurant openings in the neighborhood are still not back to pre-pandemic levels. 

___________________________________________________________________________________________________________________

### Analysis Questions

Our project used the SF business location data to answer four discrete quantifiable questions about the pandemic’s impact on SF businesses, zooming in on SF restaurants and neighborhoods from 2018-2022: 
1. Has the number of annual business openings increased?
2. In which neighborhoods have occurred the most openings?
3. How have restaurant openings changed throughout the course of the pandemic?
4. How have restaurant openings changed in the three neighborhoods with the most restaurants?

___________________________________________________________________________________________________________________

In [10]:
import pandas as pd
import altair as alt

In [11]:
# Import dataframes that were created in "data vetting" notebook
sf_biz_18_22 = pd.read_csv("sf_biz_18_22.csv", low_memory=False)
sf_biz_industry = pd.read_csv("sf_biz_industry.csv", low_memory=False)

#### Q1: Has the number of annual business openings increased?

In [3]:
# Create pivot table to show the number of openings by year and store in new dataframe ("sf")
sf = sf_biz_18_22["Start_Year"].value_counts().reset_index()
sf

,index,Start_Year
0,2018,19424
1,2019,18918
2,2021,14930
3,2020,14051
4,2022,13448


In [18]:
sf_month_year = pd.to_datetime(sf_biz_18_22['Location Start Date']).dt.strftime('%Y-%m').value_counts().reset_index().head()
sf_month_year

,index,Location Start Date
0,2018-01,3299
1,2019-01,3174
2,2020-01,2658
3,2021-01,2103
4,2022-01,2015


In [19]:
# Rename column headers of newly created dataframe
sf_month_year = sf_month_year.rename(columns={"index": "Start_Month_Year", "Location Start Date": "Count_Openings"})

In [21]:
# Rename column headers of newly created dataframe
sf = sf.rename(columns={"index": "Start_Year", "Start_Year": "Count_Openings"})

In [22]:
# Check if data storage was successful
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Start_Year      5 non-null      int64
 1   Count_Openings  5 non-null      int64
dtypes: int64(2)
memory usage: 208.0 bytes


In [26]:
sf["Start_Year"] = sf["Start_Year"].astype(str) 

In [22]:
# YEAR ONLY: Visualize findings in a bar graph
chart = alt.Chart(sf).mark_bar(size=60).encode(
    x=alt.X('Start_Year:O', axis=alt.Axis(title='Year', format='d', labelAngle=0), scale=alt.Scale(padding=0.01)),
    y=alt.Y('Count_Openings:Q', sort='-y', axis=alt.Axis(title='Number of Openings'), scale=alt.Scale(domain=[0, 22000])),
    tooltip=[alt.Tooltip('Count_Openings:Q', format='.0f')]
).properties(
    title='Business Openings in San Francisco (2018-2022)',
    width=500
).configure_mark(
    opacity=0.4,
    color='darkturquoise'
    )

# display chart
chart

alt.Chart(...)

In [23]:
# MONTH-YEAR: Visualize findings in a line chart
chart = alt.Chart(sf).mark_line().encode(
    x=alt.X('Start_Year:O', axis=alt.Axis(title='Year', format='d', labelAngle=0), scale=alt.Scale(padding=0.01)),
    y=alt.Y("Count_Openings", sort="-x", axis=alt.Axis(title='Number of Openings'), scale=alt.Scale(domain=[0, 22000])),
).properties(
    title="Business Openings in San Francisco (2018-2022)",
    width=500
)

# display the chart
chart

alt.Chart(...)

A1: Overall, the number of annual business openings has decreased in the past five years.

___________________________________________________________________________________________________________________

#### Q2: In which neighborhoods have occurred the most openings in the last five years?

In [36]:
# Check if neighborhoods are complete
len(sf_biz_18_22["Hoods"].unique())

42

In [41]:
# Create pivot table showing number of business openings by neighborhood; sort in descending to have neighborhoods with most openings at the top
sf_biz_18_22["Hoods"].value_counts().sort_values(ascending = False).head(10)

Financial District/South Beach    9732
South of Market                   5220
Mission                           5099
Bayview Hunters Point             3484
Sunset/Parkside                   3087
Marina                            2100
Castro/Upper Market               2075
Tenderloin                        1944
Outer Richmond                    1929
Chinatown                         1864
Name: Hoods, dtype: int64

In [42]:
# Create filtered dataframe of top five neighborhoods by openings
sf_biz_hood_bar = sf_biz_18_22["Hoods"].value_counts().sort_values(ascending = False).reset_index().head()

In [94]:
# Rename column headers of newly created dataframe
sf_biz_hood_bar = sf_biz_hood_bar.rename(columns={"index": "Hood", "Hoods": "Count_Openings"})

In [91]:
# Review if renaming was successful
sf_biz_hood_bar.head()

,Hood,Count_Openings
0,Financial District/South Beach,9732
1,South of Market,5220
2,Mission,5099
3,Bayview Hunters Point,3484
4,Sunset/Parkside,3087


In [140]:
# Visualize findings in a bar graph
chart = alt.Chart(sf_biz_hood_bar).mark_bar(size=25).encode(
    y=alt.Y('Hood', sort='-x', axis=alt.Axis(title='')),
    x=alt.X('Count_Openings', axis=alt.Axis(title='Number of Openings', format='d', labelAngle=0), scale=alt.Scale(padding=0.01)),
    tooltip=[alt.Tooltip('Count_Openings', format='.0f')]
).properties(
    title='Five Neighborhoods with Most Business Openings (2018-2022)',
    width=500,
    height=200
).configure_mark(
    opacity=0.4,
    color='darkturquoise'
    )

# display the chart
chart

alt.Chart(...)

In [158]:
sf_biz_hood = sf_biz_18_22.groupby(["Hoods", "Start_Year"]).size().reset_index()

In [159]:
sf_biz_hood.rename(columns={0: "Count_Openings"}, inplace=True)

In [160]:
sf_biz_hood.head()

,Hoods,Start_Year,Count_Openings
0,Bayview Hunters Point,2018,760
1,Bayview Hunters Point,2019,722
2,Bayview Hunters Point,2020,731
3,Bayview Hunters Point,2021,670
4,Bayview Hunters Point,2022,601


In [161]:
condition = (sf_biz_hood["Hoods"] == "Bayview Hunters Point") | (sf_biz_hood["Hoods"] == "Mission") | (sf_biz_hood["Hoods"] == "Sunset/Parkside") | (sf_biz_hood["Hoods"] == "South of Market") | (sf_biz_hood["Hoods"] == "Financial District/South Beach")

In [162]:
sf_biz_hood_line = sf_biz_hood[condition]

In [163]:
sf_biz_hood_line.head()

,Hoods,Start_Year,Count_Openings
0,Bayview Hunters Point,2018,760
1,Bayview Hunters Point,2019,722
2,Bayview Hunters Point,2020,731
3,Bayview Hunters Point,2021,670
4,Bayview Hunters Point,2022,601


In [164]:
# Make Year and Hoods variable a string to display properly
sf_biz_hood_line['Start_Year'] = sf_biz_hood_line['Start_Year'].astype(str)
sf_biz_hood_line['Hoods'] = sf_biz_hood_line['Hoods'].astype(str)

/tmp/ipykernel_18477/3006231422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sf_biz_hood_line['Start_Year'] = sf_biz_hood_line['Start_Year'].astype(str)
/tmp/ipykernel_18477/3006231422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sf_biz_hood_line['Hoods'] = sf_biz_hood_line['Hoods'].astype(str)


In [165]:
sf_biz_hood_line['Hoods'].unique()

array(['Bayview Hunters Point', 'Financial District/South Beach',
       'Mission', 'South of Market', 'Sunset/Parkside'], dtype=object)

In [167]:
# Visualize top five neighborhoods by openings over the past five years
alt.Chart(sf_biz_hood_line).mark_line().encode(
    x=alt.X('Start_Year:O', axis=alt.Axis(title='Year', format='d', labelAngle=0, titleFontSize=14), scale=alt.Scale(padding=0.01)),
    y=alt.Y("Count_Openings", axis=alt.Axis(title='Number of Openings', titleFontSize=14), sort="-x"),
    color="Hoods",
    tooltip=[alt.Tooltip('Count_Openings', format='.0f')]
).properties(
    title=alt.TitleParams("Five Neighborhoods with Most Business Openings (2018-2022)", fontSize=16),
    width=500
).configure_legend(
    labelFontSize=11,
    titleFontSize=14
).configure(
    line={
        'strokeWidth': 2,  # change the line thickness
        'opacity': 0.7,  # change the line opacity
    }
)

alt.Chart(...)

A2: Most business openings have occurred in Financial District/South Beach, South of Market, Mission, Bayview Hunters Point and Sunset/Parkside.

___________________________________________________________________________________________________________________

#### Q3: How have restaurant openings changed in the past five years?

In [173]:
sf_biz_industry["LIC Code"].value_counts().head(10)

H25    537
H24    516
J02    337
H26    213
H36    144
H07    135
H34    130
H85    122
H74    118
Hhh    105
Name: LIC Code, dtype: int64

In [174]:
sf_biz_industry.loc[sf_biz_industry["LIC Code"].str.contains("H24|H25|H26"), "LIC Code"] = "restaurant"

In [175]:
# Consolidate all restaurant business locations by size (SMALL = LIC Code "H24", MEDIUM = LIC Code "H25", LARGE = LIC Code "H26")
# sf_biz_industry["LIC Code Clean"] = sf_biz_industry["LIC Code"].apply(lambda x: "H24" if "H24" in x else ("H25" if "H25" in x else ("H26" if "H26" in x else x)))

In [176]:
# Consolidate LARGE restaurant business locations (LIC Code "H26")
sf_biz_industry["LIC Code Clean"] = sf_biz_industry["LIC Code"].apply(lambda x: "H26" if "H26" in x else x)

In [223]:
sf_biz_industry["LIC Code"].value_counts().head(10)

restaurant    4149
H68            812
Pos01          648
J02            529
Hhh            520
D04            313
D19            198
H07            187
H86            175
H34            172
Name: LIC Code, dtype: int64

In [177]:
restaurant = (sf_biz_industry["LIC Code"] == "restaurant") 

In [179]:
sf_biz_restaurant = sf_biz_industry[restaurant]

In [180]:
sf_biz_restaurant_bar = sf_biz_restaurant.groupby(["LIC Code", "Start_Year"]).size().reset_index()

In [181]:
sf_biz_restaurant_bar.rename(columns={0: "Count_Openings"}, inplace=True)

In [183]:
sf_biz_restaurant_bar["Start_Year"] = sf_biz_restaurant_bar["Start_Year"].astype(str)

In [184]:
sf_biz_restaurant_bar.head()

,LIC Code,Start_Year,Count_Openings
0,restaurant,2018,362
1,restaurant,2019,366
2,restaurant,2020,251
3,restaurant,2021,298
4,restaurant,2022,279


In [189]:
# Visualize findings in a bar graph
chart = alt.Chart(sf_biz_restaurant_bar).mark_bar(size=50).encode(
    x=alt.X('Start_Year:O', axis=alt.Axis(title='Year', format='d', labelAngle=0), scale=alt.Scale(padding=0.01)),
    y=alt.Y('Count_Openings:Q', sort='-y', axis=alt.Axis(title='Number of Openings'), scale=alt.Scale(domain=[0, 450])),
    tooltip=[alt.Tooltip('Count_Openings:Q', format='.0f')]
).properties(
    title='Restaurant Openings in San Francisco (2018-2022)',
    width=500
).configure_mark(
    opacity=0.2,
    color='red'
    )

# display chart
chart

alt.Chart(...)

A3: Restaurant openings have declined overall in the past five years. After a significant drop between 2019 and 2020, restaurants rebounded briefly in 2021 before declining again in 2022.

___________________________________________________________________________________________________________________

#### Q4: How have restaurant openings changed in the past five years by neighborhood?

In [190]:
sf_biz_restaurant["Hoods"].value_counts()

Financial District/South Beach    179
Mission                           166
Tenderloin                        128
South of Market                    89
Chinatown                          80
Sunset/Parkside                    71
Marina                             70
Outer Richmond                     68
Hayes Valley                       55
Mission Bay                        54
Nob Hill                           53
Russian Hill                       46
North Beach                        46
Inner Richmond                     41
Inner Sunset                       37
Castro/Upper Market                36
Bernal Heights                     32
Lakeshore                          31
West of Twin Peaks                 27
Japantown                          25
Western Addition                   24
Haight Ashbury                     24
Noe Valley                         22
Pacific Heights                    20
Potrero Hill                       19
Excelsior                          18
Bayview Hunt

Describe why we choose to use the top 3: Financial District/South Beach, Mission and Tenderloin.

In [191]:
hood = (sf_biz_restaurant.Hoods == "Financial District/South Beach") | (sf_biz_restaurant.Hoods == "Mission") | (sf_biz_restaurant.Hoods == "Tenderloin")

In [192]:
sf_biz_restaurant_hood = sf_biz_restaurant[hood]

In [196]:
sf_restaurant_hood_line = sf_biz_restaurant_hood.groupby(["Hoods", "Start_Year"]).size().reset_index()

In [197]:
sf_restaurant_hood_line = sf_restaurant_hood_line.rename(columns={0:"Count_Openings"})

In [198]:
sf_restaurant_hood_line["Start_Year"] = sf_restaurant_hood_line["Start_Year"].astype(str)

In [205]:
# Visualize top five neighborhoods by openings over the past five years
chart = alt.Chart(sf_restaurant_hood_line).mark_line().encode(
    x=alt.X('Start_Year:O', axis=alt.Axis(title='Year', format='d', labelAngle=0, titleFontSize=14), scale=alt.Scale(padding=0.01)),
    y=alt.Y("Count_Openings", axis=alt.Axis(title='Number of Openings', titleFontSize=14), sort="-x"),
    color="Hoods",
    tooltip=[alt.Tooltip('Count_Openings', format='.0f')]
).properties(
    title=alt.TitleParams("Restaurant Openings in Financial District, Mission & Tenderloin (2018-2022)", fontSize=15),
    width=500
).configure_legend(
    labelFontSize=11,
    titleFontSize=14
).configure(
    line={
        'strokeWidth': 2,  # change the line thickness
        'opacity': 0.7,  # change the line opacity
    }
)

# display chart
chart

alt.Chart(...)

A4: Tenderloin experienced an increase in restaurants in the past five years, while Financial District/South Beach and Mission saw an overall decrease. However, restaurant openings in the Financial District have steadily increased since 2020, but still are not back to pre-pandemic levels.